1. Using the plotly and dash libraries, create a dashboard consisting of two visualizations:

- column chart that shows the largest values of the total number of vaccinations . for n selected countries (e.g., when n = 5 the chart shows the 5 countries with the most cases of disease, in descending order)
- column chart that shows the largest values of the ratio of the total number of vaccinations to the population of a country, for n selected countries
2. For calculations, use the data from the last day in the provided dataset.
3. The dashboard should consist of:
- title
- title (label) of the parameter
- parameter that takes values from 5 to 20 (in increments of 5), and determines the number of countries shown in the charts.
- two visualizations from the first point, placed side-by-side
4. To run the dash application in Google Colab, use the solution presented in the course, based on the threading and google.colab libraries. Use the function that displays the application in a new browser tab.1.

In [1]:
# Install seaborn, pandas, matplotlib, plotly, dash, and threading if necessary
!pip install pandas plotly dash

In [2]:
# Import the libraries with aliases
import dash
import pandas as pd
import seaborn as sns
import plotly.express as px
import threading
from dash import dcc, html, Input, Output
from google.colab import output, drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [3]:
df = pd.read_csv('/drive/My Drive/FINAL PROJECTS/Covid_19/project_1_python.csv')
df.head(2)

,index,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
0,0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39835428.0,18.6,1803.987,64.83,33.0,65.0
1,1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39835428.0,18.6,1803.987,64.83,33.0,65.0


In [4]:
latest_day_data = df[df['date'] == df['date'].max()].fillna(0)
print(latest_day_data.to_string())


         index iso_code      continent                          location        date  total_cases  new_cases  total_deaths  new_deaths  hosp_patients  total_tests  new_tests  tests_per_case  total_vaccinations  people_vaccinated  people_fully_vaccinated  total_boosters  new_vaccinations    population  median_age  gdp_per_capita  life_expectancy  latitude  longitude
845        845      AFG           Asia                       Afghanistan  2022-06-18     181574.0       40.0        7713.0         0.0            0.0          0.0        0.0             0.0        0.000000e+00       0.000000e+00                      0.0             0.0               0.0  3.983543e+07        18.6        1803.987            64.83   33.0000    65.0000
1690      1690      ALB         Europe                           Albania  2022-06-18     277141.0        0.0        3497.0         0.0            0.0          0.0        0.0             0.0        0.000000e+00       0.000000e+00                      0.0           

In [5]:
# Define the layout of the dashboard
app = dash.Dash()

app.layout = html.Div([
    html.H1("Vaccination Dashboard"),
    html.Br(),
    html.Label("Select the number of countries to display:"),
    dcc.Slider(
        id='country-slider',
        min=5,
        max=20,
        step=5,
        marks={i: str(i) for i in range(5, 21, 5)},
        value=5
    ),

    html.Div(id='charts-container', style={'display': 'flex'}),
])



In [6]:
# Define the callback to update the charts based on the slider value
@app.callback(
    Output('charts-container', 'children'),
    [Input('country-slider', 'value')]
)
def update_charts(selected_countries):
    # Sort data by total vaccinations and select top n countries
    top_vaccinated_countries = latest_day_data.sort_values(by='people_fully_vaccinated', ascending=False).head(selected_countries)

    # Sort data by vaccination ratio and select top n countries
    top_vaccination_ratio_countries = latest_day_data.assign(vaccination_ratio = latest_day_data['people_fully_vaccinated'] / latest_day_data['population']).sort_values(by='vaccination_ratio', ascending=False).head(selected_countries)

    # Create first chart for total vaccinations with Viridis color scale
    fig1 = px.bar(top_vaccinated_countries, x='location', y='people_fully_vaccinated', title=f'Top {selected_countries} Countries by Total Vaccinations',
              color='people_fully_vaccinated', color_continuous_scale='Viridis')

    # Create second chart for vaccination ratio with Viridis color scale
    fig2 = px.bar(top_vaccination_ratio_countries, x='location', y='vaccination_ratio', title=f'Top {selected_countries} Countries by Vaccination Ratio',
              color='vaccination_ratio', color_continuous_scale='Viridis')

    # Return the charts side by side
    return [
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2)
]

In [7]:
# Start the Dash app in a separate thread
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>